<a href="https://colab.research.google.com/github/dssg/triage/blob/master/example/colab/colab_triage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Triage Tutorial

## Problem Overview

This notebook provides a quick, interactive tutorial for [triage](http://www.datasciencepublicpolicy.org/our-work/tools-guides/triage/), a python machine learning pipeline for social good problems, using a sample of the data provided by [DonorsChoose](https://www.donorschoose.org/) to the [2014 KDD Cup](https://www.kaggle.com/c/kdd-cup-2014-predicting-excitement-at-donors-choose/data). Public schools in the United States face large disparities in funding, often resulting in teachers and staff members filling these gaps by purchasing classroom supplies out of their own pockets. DonorsChoose is an online crowdfunding platform that tries to help fill this gap by allowing teachers to seek funding for projects and resources from the community (projects can include classroom basics like books and markers, larger items like lab equipment or musical instruments, specific experiences like field trips or guest speakers). Projects on DonorsChoose expire after 4 months, and if the target funding level isn't reached, the project receives no funding. Since its launch in 2000, the platform has helped fund over 2 million projects at schools across the US, but about 1/3 of the projects that are posted nevertheless fail to meet their goal and go unfunded.

For the purposes of this tutorial, we'll imagine that DonorsChoose has hired a digital content expert who will review projects and help teachers improve their postings and increase their chances of reaching their funding threshold. Because this individualized review is a labor-intensive process, the digital content expert has time to review only 10% of the projects posted to the platform on a given day. 

### The Modeling Problem

You are a data scientist working with DonorsChoose, and your task is to help this content expert focus their limited resources on projects that most need the help. As such, you want to build a model to identify projects that are least likely to be fully funded before they expire and pass them off to the digital content expert for review.

In building that model, our unit of analysis (what triage calls a **cohort**) will be new projects right at the time they're posted, while the **label** we're seeing to predict is whether or not the project reaches its funding goal in the subsequent 4 months (before it expires), making our task a binary classification problem. In order to make this prediction, we'll develop **features** that include information we know about the project when it's posted as well as historical performance of other projects posted by this teacher, school, etc.

### Outline of the Tutorial

The remainder of this tutorial will focus on how to use `triage` to solve this problem. Starting from scratch, we'll:
- **Install our tools**, including triage and a postgres server.
- **Explore the data** to get familiar with its structure.
- **Formulate the project** to make sure the models we build meet the needs of the context (and see how to configure `triage` along the way).
- **Build models**, using `triage` to run the modeling pipeline.
- **Look at the results** to ensure they make sense.
- **Select the model to deploy** using the `audition` component of `triage`.
- **Audit our models for bias** using `aequitas`.
- **Lear about next steps** and where to go from here.



## Getting Set Up

We'll need a few dependencies to run triage in a colab notebook:
- A local postgresql server (we'll use version 11)
- A simplified dataset loaded into this database (we'll use data from DonorsChoose)
- Triage and its dependencies (we'll use the current version in pypi)

In [ ]:
# Install and start postgresql-11 server
!sudo apt-get -y -qq update
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo apt-key add -
!echo "deb http://apt.postgresql.org/pub/repos/apt/ `lsb_release -cs`-pgdg main" |sudo tee  /etc/apt/sources.list.d/pgdg.list
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-11 postgresql-client-11
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `donors_choose` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS donors_choose;'

!sudo -u postgres psql -U postgres -c 'CREATE DATABASE donors_choose;'

# Environment variables for connecting to the database
%env DEMO_DATABASE_NAME=donors_choose
%env DEMO_DATABASE_HOST=localhost
%env DEMO_DATABASE_PORT=5432
%env DEMO_DATABASE_USER=postgres
%env DEMO_DATABASE_PASS=postgres

In [ ]:
# Download sampled DonorsChoose data and load it into our postgres server
!curl -s -OL https://dsapp-public-data-migrated.s3.us-west-2.amazonaws.com/donors_sampled_20210920_v3.dmp
!PGPASSWORD=$DEMO_DATABASE_PASS pg_restore -h $DEMO_DATABASE_HOST -p $DEMO_DATABASE_PORT -d $DEMO_DATABASE_NAME -U $DEMO_DATABASE_USER -O -j 8 donors_sampled_20210920_v3.dmp

In [ ]:
# Install triage and its dependencies
%%capture
!pip install triage

🛑  &nbsp;&nbsp;**WARNING! Be sure to restart the session before moving on to the next section! Otherwise, Colab won't recognize that `triage` has been installed. To restart the session, go to `Runtime` menu and select the option `Restart session`.**

## A Quick Look at the DonorsChoose Data

Before getting into triage, let's just take a quick look at the data we'll be using here. To get started, we'll need to connect to the database we just created...

In [ ]:
from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
import pandas as pd

db_url = URL(
            'postgres',
            host='localhost',
            username='postgres',
            database='donors_choose',
            password='postgres',
            port=5432,
        )

db_engine = create_engine(db_url)

The DonorsChoose dataset contains four main tables we'll need here:
- **Projects** contains information about each project as well as some details about the teacher posting it and their school and district
- **Essays** contains the detailed descriptions that the teacher post describing their project and needs
- **Resources** contains detailed information about the specific number, type, and cost of resources being asked for in the project
- **Donations** contains information about the donations received by each project on a transactional level, as well as some details about the donor

Let's take a look at the projects:

In [ ]:
pd.read_sql('SELECT COUNT(*) FROM data.projects', db_engine)

In [ ]:
pd.read_sql('SELECT * FROM data.projects LIMIT 5', db_engine)

Note that the `projectid_str` column can be used to link out to the other tables. For instance, let's look at what we can find out about project `30c034618e67d00c641f9b5b7775c0f4`:

In [ ]:
pd.read_sql("SELECT * FROM data.essays WHERE projectid_str='30c034618e67d00c641f9b5b7775c0f4'", db_engine)

In [ ]:
pd.read_sql("""
  SELECT project_resource_type,
        COUNT(*) AS num_distinct_resources, 
        SUM(item_quantity) AS num_total_resources,
        AVG(item_unit_price) AS avg_price,
        SUM(item_unit_price * item_quantity) AS total_cost
  FROM data.resources 
  WHERE projectid_str='30c034618e67d00c641f9b5b7775c0f4'
  GROUP BY 1;
  """, db_engine)

In [ ]:
pd.read_sql("""
  SELECT 
    COUNT(*) AS num_donations,
    SUM(donation_to_project) AS total_donation,
    SUM(CASE WHEN is_teacher_acct THEN 1 ELSE 0 END) AS num_teacher_donation
  FROM data.donations 
  WHERE projectid_str='30c034618e67d00c641f9b5b7775c0f4'
  ;
  """, db_engine)

## Formulating the project

Now that we're familiar with the available data, let's turn to the prediction problem at hand. Because reviewing and offering suggestions to posted projects will be time and resource-intensive, we might assume that DonorsChoose can only help a fraction of all projects that get posted, let's suppose 10%. Then, we might formulate our problem along the lines of:

**Each day, for all the projects posted on that day, can we identify the 10% of projects with the highest risk of not being fully funded within 4 months to prioritize for review by digital content experts.**

With this formulation in mind, we can define a cohort and label for our analysis. `triage` will allow us to define these directly as a SQL query, so let's start there...

### Defining the Cohort and Label

Because most models to inform important decisions will need to generalize into the future, `triage` focuses on respecting the temporal nature of the data (discussed in more detail below). The `cohort` is the set of relevant entities for model training/prediction at a given point in time, which `triage` referrs to as an `as_of_date`.

🚧 &nbsp;&nbsp;NOTE: In `triage`, an `as_of_date` is taken to be midnight at the **beginning** of that date.

Here, the cohort is relatively straightforward: we simply want to identify all of the projects that were posted, right on the day of posting. Although we were looking at the identifier `projectid_str` above, `triage` looks for a column called `entity_id` to uniquely identify entities to its models. We've already added this column to this dataset, so we'll use that below.

🚧 &nbsp;&nbsp;NOTE: `triage` expects entities in the data to be identified by an **integer column** called `entity_id`.

For modeling, we also need to consider the outcome we care about. Returning to our formulation, we described trying to identify projects which will not be fully funded within the four months they are active on the platform.

As with the cohort, notice that labels are calculated relative to a given point in time (the `as_of_date` described above) and over a specific time horizon (here, 4 months from posting). In triage, this time horizon is referred to as a `label_timespan` and is also available as a parameter to your label definition.

With those details in mind, let's look at an example of how we might define the cohort and label from our data for this project:

In [ ]:
example_as_of_date = '2012-08-07'
example_label_timespan = '4month'

pd.read_sql("""
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT 
        c.entity_id, 
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d 
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp 
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE 
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome  
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)
  ;
  """.format(as_of_date=example_as_of_date, label_timespan=example_label_timespan), db_engine)

Here, we start by defining our cohort as described above, then find all the donations to those projects within the label timespan (e.g., the following 4 months after posting), and finally compare that to the total price of the project to create a binary classification label for whether or not the project was fully funded.

Notice here that because we will intervene on projects at risk for **NOT** being fully funded, we define this as our class 1 label while those that do reach their funding goal are given class 0.

In `triage` we'll be able to use `{as_of_date}` as a placeholder for time just as we're doing here. Also note that because the `as_of_date` is taken to be midnight, we're looking at the projects posted the previous day (hence subtracting the 1 day interval in the query).

For `triage`, we use a yaml format for configuration (described further below) and we'll be able to provide this query directly:
```
label_config:
  query: |
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT 
        c.entity_id, 
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d 
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp 
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE 
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome  
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)

  name: 'fully_funded'
```

For more details these two pieces of the modeling pipeline, see the [cohort and label deep dive in the triage docs](https://dssg.github.io/triage/experiments/cohort-labels/). 

### Dealing with Time

As noted above, `triage` is designed for problems where the desire to generalize to future data and therefore is careful to respect the temporal nature of the problem. This is particularly salient in two places: defining the validation strategy for model evaluation and ensuring that features only make use of information available at the time of analysis/prediction.

For validation, the idea is generally simple: models should be trained on historical data and validated on future data. As such, `triage` constructs validation splits that reflect this process by using a certain point in time as the cut-off between training and validation and then moving this cut-off back through the data to generate multiple splits. The implementation is a bit more complicated and relies on several parameters, the details of which we won't go deep into here, but you can find a much deeper discussion in the [longer "dirty duck" tutorial](https://dssg.github.io/triage/dirtyduck/triage_intro/) as well as in the [experiment config docs](https://dssg.github.io/triage/experiments/experiment-config/).

![temporal figure](https://dssg.github.io/triage/experiments/temporal_config_graph.png)

In short, these parameters are (illustrated across three training/validation splits in the figure above):
- feature start/end times: what range of history is feature information available for?
- label start/end times: what range of history is outcome (label) data available for?
- model update frequency: what is the interval between refreshes of the model?
- test durations: over what time period will the model be in use for making predictions?
- max training history: how much historical data should be used for model training (that is, for rows/examples)?
- training/test as_of_date frequencies: within a training or validation (test) set, how frequently should cohorts be sampled?
- training/test label timespans: over what time horizon are labels (outcomes) collected?

As with the cohorts and labels, these parameters are specified to `triage` via its yaml configuration file. Here's what this will look like for our setting:
```
temporal_config:

    # first date our feature data is good
    feature_start_time: '2000-01-01'
    feature_end_time: '2013-06-01'

    # first date our label data is good
    # donorschoose: as far back as we have good donation data
    label_start_time: '2011-09-02'
    label_end_time: '2013-06-01'

    model_update_frequency: '4month'

    # length of time defining a test set
    test_durations: ['3month']
    # defines how far back a training set reaches
    max_training_histories: ['1y']

    # we sample every day, since new projects are posted
    # every day
    training_as_of_date_frequencies: ['1day']
    test_as_of_date_frequencies: ['1day']
    
    # when posted project timeout
    label_timespans: ['3month']
```

### Model Evaluation Metrics

The temporal configuration described above will create several training and validation splits that can be used to estimate the generalization performance of your models and select a model specification to use going forward. In order to do so, of course, you need to choose an appropriate metric (or metrics) by which to evaluate your models. `triage` can use any of the metrics specified by `sklearn` and in general you'll want to focus on those that best reflect the goals, constraints, and deployment scenario of your project. For instance, in our example project, DonorsChoose can help only 10% of the projects posted to the site, so a metric like precision in the top 10% would reflect how efficiently these limited resources are being allocated to projects that would not be fully funded without additional support.

Although we might want to focus on `precision@10%` as our primary metric, often it can be helpful to look at both precision and recall at a range of thresholds (both percentiles and absolute numbers) both for the purposes of debugging and understanding how sensitive your results are to the available resources, describing a "menu" of policy choices.

The `scoring` section of the yaml configuration file allows you specify separate evaluation metrics for both the training and validation set results, indicating both the type of metric (e.g., `precision`, `recall`, etc) and, where needed, the thresholds at which to calculate them. Here's what that looks like for our example project:

```
scoring:
    testing_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]

    training_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]
```

### Defining Features

Feature generation is typically the most important aspect of how well your machine learning models will work, so `triage` provides considerable flexibility for feature definition. However, this also means that this section of the configuration file can be particularly complicated and may require some experimentation to get familiar with. A few resources may be helpful for a deeper look at how features work in `triage`:
- [Feature Definition in the Quickstart Guide](https://dssg.github.io/triage/triage_project_workflow/#define-some-additional-features)
- [Feature Generation in the triage Documentation](https://dssg.github.io/triage/experiments/experiment-config/#feature-generation)
- [Features in the Example Configuration File](https://github.com/dssg/triage/blob/master/example/config/experiment.yaml#L102)

Features in `triage` are defined in blocks, grouping together features drawn from the same data source and allowing several related features to be constructed in a very compact format. Each of these blocks is a list item under the `feature_aggregations` section of your yaml configuration file and contains the following information:
- A `prefix` that is used to identify the group of features.
- A `from_obj` that specifies the underlying information used to construct the features in this group. This can be either a table or a query in itself (in the later case, be sure to give it an alias) and must contain both an `entity_id` column as well as a date column indicating when the information was known, identified to the feature config as the `knowledge_date_column`.
- Information about how missing values should be imputed (see the documentation for details and available options here).
- Definitions of the feature quantities/columns themselves, specified either as `aggregates` or `categoricals`, including the `metrics` for aggregations over time (e.g., `sum`, `max`, `avg`, etc).
- Time ranges over which to calculate feature information, called `intervals` (e.g., last 6 months, last 5 years, etc.)
- A level of aggregation for feature information (`groups`) -- this will almost always be just `entity_id`.

🚧 &nbsp;&nbsp;NOTE: All features in `triage` are temporal aggregates. Just as `triage` is designed to carefully account for time in temporal cross-validation, it also does so in feature construction focusing on what information was known at training or validation time. Even features you might generally consider "static" need to be associated with a knowledge date for these purposes as well as an aggregation metric. This is also true for categoricals, which are first one-hot encoded from each instance then aggregated over the given time interval with the specified metric. For instance, if a patient has had several hospital stays with different primary diagnosis codes at each stay, a categorical feature using a `sum` aggregation would yield a count of how many stays had a given diagnosis while a `max` aggregation would provide an indicator of whether a given diagnosis was ever present. 

For aggregations of numeric features, the resulting feature names will have the format: 
`{prefix}_entity_id_{interval}_{quantity}_{metric}`

For categoricals, the feature names will include each categorical value after one-hot encoding:
`{prefix}_entity_id_{interval}_{quantity}_{value}_{metric}`

🚧 &nbsp;&nbsp;WARNING: Because `triage`'s features are stored in a `postgres` database, this naming convention can sometimes run afoul of the database's 63 character limit for column names, leading to truncation. When this happens, you might encounter errors indicating a given feature column appears to be missing. This can be common with categoricals with particularly long values, so recoding can be useful in those cases (as can choosing shorter prefix names).

For illustrative purposes here, we'll start with a single feature group including one categorical and continuous aggregate feature: the primary resource type for the project and the amount being asked for. Because these are both specified once at project posting time, we simply aggegate them over all time (that is, using `all` for our `interval`). Here's how we specify this in our feature configuration:

```
feature_aggregations:
  -
    prefix: 'project_features'
    from_obj: 'data.projects'
    knowledge_date_column: 'date_posted'

    aggregates_imputation:
      all:
        type: 'zero'

    categoricals_imputation:
      all:
        type: 'null_category'          

    categoricals:
      -
        column: 'resource_type'
        metrics:
          - 'max' 
        choice_query: 'select distinct resource_type from data.projects'
    
    aggregates:
      -
        quantity: 'total_asking_price'
        metrics:
          - 'sum'
      
    # Since our time-aggregate features are precomputed, feature interval is 
    # irrelvant. We keep 'all' as a default.
    intervals: ['all'] 
    groups: ['entity_id']
```

### Model and Hyperparameter Grid

You specify the types of models you want to explore, along with their hyperparameters, in the `grid_config` section of your yaml configuration file. Because there's generally no way to know a priori what model specification will work best for a given problem, `triage` makes it easy to run and explore an extensive grid by providing lists of values for each hyperparameter and training models for the full cross-product of these values.

Currently, `triage` can work with any classifiction method with an `sklearn`-style interface. In addition to machine learning algorithms found in standard packages, `triage` includes a couple of built-in methods you might find useful:
- `ScaledLogisticRegression` wraps the `sklearn`  logistic regression with a min-max scaler to ensure that the input features are on the same scale for regularization. It accepts the same hyperparameters as the underlying `sklearn` method.
- `BaselineRankMultiFeature` is a simple baseline method that ranks examples by one or more features, replicating a comonsense approach that could be taken without making use of machine learning. This method takes a single hyperparameter, `rules`, specified as a list of dictionaries with the keys `feature` and `low_value_high_score` to specify the directin of the ranking. Examples are sorted first by the first feature in this list, then the next, and so on.
- `SimpleThresholder` is another basic baseline method, allowing you to specify a heuristic, rule-based approach to classifying examples. It uses two hyperparameters: a list of `rules` (e.g., `feature_1 > 3`) and a `logical_operator` (e.g., `and` or `or`) to specify how the rules are combined.

To specify a model type in your grid config, you use the model's class path as a key and each hyperparameter as a key another level down. For example:
```
'module.submodule.ClassName':
    param_1: [1,3,5,10,20]
    param_2: [100, 500, 1000]
```

For our purposes here, we'll start with a very small grid that can run quickly in a colab notebook. Here's how that will look:
```
grid_config:
    'sklearn.ensemble.RandomForestClassifier':
        n_estimators: [150]
        max_depth: [50]
        min_samples_split: [25]
    
    'sklearn.tree.DecisionTreeClassifier':
        max_depth: [3]
        max_features: [null]
        min_samples_split: [25]
      
    'triage.component.catwalk.estimators.classifiers.ScaledLogisticRegression':
        C: [0.1]
        penalty: ['l1']
    
    'triage.component.catwalk.baselines.rankers.BaselineRankMultiFeature':
        rules:
            - [{feature: 'project_features_entity_id_all_total_asking_price_sum', low_value_high_score: False}]
```

### Auditing Models for Bias

The final section of the configuration file specifies how you want to evaluate your models for bias and fairness using the [aequitas](http://www.datasciencepublicpolicy.org/our-work/tools-guides/aequitas/) toolkit. In order to do so, you need to tell `triage` what attributes are relavent for bias audits, a table or SQL query to specify these attributes (the `from_obj`), and the reference group to calculate disparities relative to (the value for this group will serve as the denominator for disparity calculations. Like the evaluation metrics described above, you'll also need to specify the set of thresholds against which you want to calculate fairness metrics. Note that `aequitas` will calculate the full range of confusion matrix-derived disparity metrics for all of your models, allowing you to explore how your models perform under different conceptualizations of fairness.

To illustrate the use of a bias audit in our example project, we'll look at the `teacher_prefix` attribute as a proxy for the sex of the teacher, using `Mr.` as a reference group. Note that the `from_obj_table` will be joined using an `entity_id` and `as_of_date`, so you must specify a `knowledge_date_column` in the config, as some attributes (or your knowledge of them) might change over time. `aequitas` will use the most recent value of the attribute it finds for a given entity prior to the specified `as_of_date`. Here's how we turn that into a section in our configuration yaml:

```
bias_audit_config:
    from_obj_table: 'data.projects'
    attribute_columns:
        - 'teacher_prefix'
    knowledge_date_column: 'date_posted'
    entity_id_column: 'entity_id'
    ref_groups_method: 'predefined'
    ref_groups:
        'teacher_prefix': 'Mr.'
    thresholds:
        percentiles: [5, 10, 15, 20, 25, 50, 100]
        top_n: [25, 50, 100]
```

## Running Triage

Now that we've walked through the various aspects of configuring triage, we're ready to run our model grid! In order to do so, we need three pieces:
- Our configuration file, pulling together the elements described above into a single yaml file we'll call `experiment_config.yaml` (in `triage`, an "experiment" is a run with a set of parameters and model types).
- Credentials for connecting to your database, stored in a configuration file called `database.yaml` (alternatively, you can specify them through environment variables)
- Code to run your `triage` experiment. This can be done via either a command line tool or python interface, the latter of which provides more flexibility so we'll focus on that approach here with a short python script called `run.py`.

The following three sections provides the contents of each of these three files for our DonorsChoose project. In a real project, of course, these would be stored as separate files on your system, but here we include them inline. The `run.py` sets up logging, connects to the database, loads your configuration file, and creates and runs a `MultiCoreExperiment` object from `triage`.

### experiment_config.yaml

In [ ]:
config_yaml = """
config_version: 'v8'

model_comment: 'triage demo'

random_seed: 1995

temporal_config:

    # first date our feature data is good
    feature_start_time: '2000-01-01'
    feature_end_time: '2013-06-01'

    # first date our label data is good
    # donorschoose: as far back as we have good donation data
    label_start_time: '2011-09-02'
    label_end_time: '2013-06-01'

    model_update_frequency: '4month'

    # length of time defining a test set
    test_durations: ['3month']
    # defines how far back a training set reaches
    max_training_histories: ['1y']

    # we sample every day, since new projects are posted
    # every day
    training_as_of_date_frequencies: ['1day']
    test_as_of_date_frequencies: ['1day']
    
    # when posted project timeout
    label_timespans: ['3month']


label_config:
  query: |
    WITH cohort_query AS (
      SELECT distinct(entity_id)
      FROM data.projects
      WHERE date_posted = '{as_of_date}'::date - interval '1day'
    )
    , cohort_donations AS (
      SELECT 
        c.entity_id, 
        COALESCE(SUM(d.donation_to_project), 0) AS total_donation
      FROM cohort_query c
      LEFT JOIN data.donations d 
        ON c.entity_id = d.entity_id
        AND d.donation_timestamp 
          BETWEEN '{as_of_date}'::date - interval '1day'
          AND '{as_of_date}'::date + interval '{label_timespan}'
      GROUP BY 1
    )
    SELECT c.entity_id,
    CASE 
      WHEN COALESCE(d.total_donation, 0) >= p.total_asking_price THEN 0
      ELSE 1
    END AS outcome  
    FROM cohort_query c
    JOIN data.projects p USING(entity_id)
    LEFT JOIN cohort_donations d using(entity_id)

  name: 'fully_funded'


feature_aggregations:
  -
    prefix: 'project_features'
    from_obj: 'data.projects'
    knowledge_date_column: 'date_posted'

    aggregates_imputation:
      all:
        type: 'zero'

    categoricals_imputation:
      all:
        type: 'null_category'          

    categoricals:
      -
        column: 'resource_type'
        metrics:
          - 'max' 
        choice_query: 'select distinct resource_type from data.projects'
    
    aggregates:
      -
        quantity: 'total_asking_price'
        metrics:
          - 'sum'
      
    # Since our time-aggregate features are precomputed, feature interval is 
    # irrelvant. We keep 'all' as a default.
    intervals: ['all']

grid_config:
    'sklearn.ensemble.RandomForestClassifier':
        n_estimators: [150]
        max_depth: [50]
        min_samples_split: [25]
    
    'sklearn.tree.DecisionTreeClassifier':
        max_depth: [3]
        max_features: [null]
        min_samples_split: [25]
      
    'triage.component.catwalk.estimators.classifiers.ScaledLogisticRegression':
        C: [0.1]
        penalty: ['l1']
    
    'triage.component.catwalk.baselines.rankers.BaselineRankMultiFeature':
        rules:
            - [{feature: 'project_features_entity_id_all_total_asking_price_sum', low_value_high_score: False}]


scoring:
    testing_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]

    training_metric_groups:
        -
          metrics: [precision@, recall@]
          thresholds:
              percentiles: [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                  10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                  20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
                  30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 
                  40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                  50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
                  60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
                  70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
                  80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
                  90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
                  100]
              top_n: [25, 50, 100]  
          
bias_audit_config:
    from_obj_table: 'data.projects'
    attribute_columns:
        - 'teacher_prefix'
    knowledge_date_column: 'date_posted'
    entity_id_column: 'entity_id'
    ref_groups_method: 'predefined'
    ref_groups:
        'teacher_prefix': 'Mr.'
    thresholds:
        percentiles: [5, 10, 15, 20, 25, 50, 100]
        top_n: [25, 50, 100]
        
individual_importance:
    methods: [] # empty list means don't calculate individual importances
    n_ranks: 1 
"""

### database.yaml

In [ ]:
database_yaml = """
host: localhost
user: postgres
db: donors_choose
pass: postgres
port: 5432
role: postgres
"""

### run.py

In [ ]:
import yaml
import shutil
import os
import logging

from sqlalchemy.engine.url import URL
from sqlalchemy.event import listens_for
from sqlalchemy.pool import Pool

from triage.util.db import create_engine
from triage.experiments import MultiCoreExperiment


def run_triage():

  # andrew_id = os.getenv('USER')
  # user_path = os.path.join('/data/users/', andrew_id)
  user_path = '/content'

  # add logging to a file (it will also go to stdout via triage logging config)
  log_filename = os.path.join(user_path, 'triage.log')
  logger = logging.getLogger('')
  hdlr = logging.FileHandler(log_filename)
  hdlr.setLevel(15)   # verbose level
  hdlr.setFormatter(logging.Formatter('%(name)-30s  %(asctime)s %(levelname)10s %(process)6d  %(filename)-24s  %(lineno)4d: %(message)s', '%d/%m/%Y %I:%M:%S %p'))
  logger.addHandler(hdlr)

  # creating database engine
  # dbfile = os.path.join(user_path, 'database.yaml')

  # with open(dbfile, 'r') as dbf:
  #     dbconfig = yaml.safe_load(dbf)

  dbconfig = yaml.safe_load(database_yaml)
  print(dbconfig['role'])

  # assume group role to ensure shared permissions
  @listens_for(Pool, "connect")
  def assume_role(dbapi_con, connection_record):
      logging.debug(f"setting role {dbconfig['role']};")
      dbapi_con.cursor().execute(f"set role {dbconfig['role']};")
      # logging.debug(f"setting role postres;")
      # dbapi_con.cursor().execute(f"set role postgres;")

  db_url = URL(
              'postgres',
              host=dbconfig['host'],
              username=dbconfig['user'],
              database=dbconfig['db'],
              password=dbconfig['pass'],
              port=dbconfig['port'],
          )

  db_engine = create_engine(db_url)

  triage_output_path = os.path.join(user_path, 'triage_output')
  os.makedirs(triage_output_path, exist_ok=True)

  # loading config file
  # with open('%s_triage_config.yaml' % andrew_id, 'r') as fin:
  #     config = yaml.safe_load(fin)

  config = yaml.safe_load(config_yaml)

  # creating experiment object
  experiment = MultiCoreExperiment(
      config = config,
      db_engine = db_engine,
      project_path = triage_output_path,
      n_processes=2,
      n_bigtrain_processes=1,
      n_db_processes=2,
      replace=True,
      save_predictions=True
      )

  # experiment.validate()
  experiment.run()

### Let's run triage!

With these three files in place, we can simply run our model grid by calling `run_triage()`. Doing so will train and validate the four model specifications described above across three temporal validation splits. The run will output a log of its progress and store results into the postgres database.

In [ ]:
run_triage()

### Checking the results 

Running `triage` will generate three types of outputs: 
1. Summary report of the experiment
2. Objects stored to disk
3. Results stored in the database.

#### 1. Summary report of the experiment 

After running an experiment in `triage` it will generate a summary report with important sanity checks to review before moving on to [Model Selection](#model-selection) and [Bias Audit](#bias-audit). Generally, this report would be generated by following this [guide](https://github.com/dssg/triage/tree/master/src/triage/component/postmodeling) using a notebook template. Since we are executing `triage` in Colab, we will call the methods of the summary report directly. 


In [ ]:
import yaml
import pandas as pd

from sqlalchemy.engine.url import URL
from triage.util.db import create_engine
from triage.component.postmodeling.experiment_summarizer import ExperimentReport
from matplotlib import pyplot as plt
%matplotlib inline

dbconfig = yaml.safe_load(database_yaml)
db_url = URL(
            'postgres',
            host=dbconfig['host'],
            username=dbconfig['user'],
            database=dbconfig['db'],
            password=dbconfig['pass'],
            port=dbconfig['port'],
        )

db_engine = create_engine(db_url)

We need to get the hash code generated for this the experiment run. We can look for the hash on the database in table `triage_metadata-triage_runs` sorted by the latest date times (more details on the section [Results stored in the database](#3-results-stored-in-the-database))

In [ ]:
q = """ 
    select run_hash, start_time 
    from triage_metadata.triage_runs 
    order by start_time desc
    limit 1
"""

pd.read_sql(q, db_engine)

For generating the Experiment Summary Report, we need to define the performance metric, threshold, bias metric, and the priority groups we would like to look for. In our Donors Choose example, since we are interested in predicting the **10% of projects with the highest risk of not being fully funded within 4 months** we will do a first check on the performance of the models for the `precision` at the `10%`. Additionally, we want to analyze recall disparity accross different teacher prefixes. 

In [ ]:
# Triage created hash(es) of the experiment(s) you are interested in. 
# It has to be a list (even if single element)
experiment_hashes = ['98112c011d842c43e841c415116ef179']

# Model Performance metric and threshold
# These default to 'recall@' and '1_pct'
performance_metric = 'precision@'
threshold = '10_pct'

# Bias metric defaults to tpr_disparity and bias metric values for all groups generated (if bias audit specified in the experiment config)
bias_metric = 'tpr_disparity'
bias_priority_groups = {'teacher_prefix': ['Dr.', 'Mr.', 'Mrs.', 'Ms.']}

# Create the Experient Rerport 
rep = ExperimentReport(
    engine=db_engine,
    experiment_hashes=experiment_hashes,
    performance_priority_metric=performance_metric,
    threshold=threshold,
    bias_priority_metric=bias_metric,
    bias_priority_groups=bias_priority_groups
)

##### Summary report of the experiment run

The summary gives a quick an short information of what happend in the experiment. 
* The number of temporal splits generated based on temporal configuration 
* The number of unique date times on those temporal splits
* The average size of the cohorts and their baserates
* The number of features generated and used in your models
* The number of feature groups 
* The number of different type of models generated, e.g., Random Forest, Decision Tree, etc.
* The number of models generated based on your grid configuration 
* The best average performance metric defined obtained and which model type generated it 
* A first glance of the disparity metric defined over the groups you have defined

If the information looks correct based on your configuration of the experiment it means that the general sanity checks were fulfilled. If not, you need to come back to your experiment setup and look for inconsistences, e.g., the start and end dates for features and labels.

In [ ]:
rep.generate_summary()

##### Visualize temporal splits  

An interactive way to see the different temporal splits generated by `triage` based on the information of the experiment configuration. In our DonorsChoose example, we defined the labels started from **September 2, 2011** to **June 1, 2013**; and a label window of **3 months**. Based on that information, the first temporal split generated by `triage` for the validation matrix ends on the last available day of May 2013 -**May 20, 2013**- and begins three months earlier on the last available day of February -**February 28, 2013**-. The train matrix then has labels that end on **February 27, 2013** and begin on **December 1, 2012**. For a deeper discussion on this topic, you can refer to [longer "dirty duck" tutorial](https://dssg.github.io/triage/dirtyduck/triage_intro/).

In [ ]:
rep.timesplits()

##### Cohorts generated

A plot with the cohort sizes (in blue) at different `as_of_date` along with the baserate for each (in red). The dotted line indicates the mean. 

In [ ]:
cohort_summary = rep.cohorts(generate_plots=True)

A summary of cohort statistics is also generated. 

In our DonorsChoose example, the average baserate is near 40%, the maximum number of projects in a cohort is of 44 whilest the minimum is just 1 project, based on the definitions or our cohort and labels.

In [ ]:
cohort_summary[['cohort_size', 'baserate']].describe()

**Cohort subsets**

In case subsets were defined on the experiment configuration a plot for each subset will be generated. 

In [ ]:
rep.subsets()

##### Predictors

This function will show the different features generated for the experiment with information related to the different metrics, time horizons and imputations made to each. 

In our DonorsChoose example, we have 2 group features: the total asking price, and the resource type.

In [ ]:
features = rep.features()

**Missing of features**

In case the features generated have missing values, this table will indicated the average, max, and min percentages of missingness of each feature. If a feature has 100% percentage it means that all of its values are missing.
In our DonorsChoose example, we don't have features with missing values.

In [ ]:
rep.feature_missingness()

##### Model groups built

Summary of the models built based on the model type, hyperparameters and temporal configurations defined for the experiment. In our DonorsChoose example, we have 4 different model groups: Random Forest Classifiers, Decision Tree Classifiers, Baseline Rankers, and Scaled Logistic Regressions. 

In [ ]:
rep.model_groups()

**All models built**

Each individual model built on the experiment (one per `as_of_date`, model type, hyperparameters, and temporal configuration). 


In [ ]:
rep.models()

##### Model Performance Overall Cohort

Performance plot with the evaluation metric defined for each model group in all `as_of_date`.

In [ ]:
evaluations = rep.model_performance()

**Model Performance on Cohort Subsets**

In [ ]:
subset_evaluations = rep.model_performance_subsets()

##### Model Performance vs Bias

Plot with the peformance of different model groups and the bias metric selected in all groups defined and plot of group sizes vs the cohort size and the base rate for each `as_of_date`. 

In our DonorsChoose example, we can tell that the teacher prefix `Mr.` has been used a the reference group, and that the Random Forest Classfier has no disparity across the different groups: Mr., Mrs., and Ms. but has less performance that other models. A deeper analysis and discussion in section [Bias Audit](#bias-audit) on this notebook.

In [ ]:
equity_metrics = rep.efficiency_and_equity()

##### Initial model selection and further analysis on best models 

By default, the report will pick the best performing model from each model type based on average performance to generate additional outputs about the developed models. We would not assume the existence of predictions at this stage. Therefore, we will not do analysis such as list comparisons, crosstabs, score distribution type stuff. 

In [ ]:
rep.get_best_hp_config_for_each_model_type()

#### 2. Objects stored in disk

Two types of objects will be stored to disk in the `project_path` specified in creating the experiment object:
- The matrices used for model training and validation, stored as CSV files and associated metadata in yaml format.
- The trained model objects themselves, stored as `joblib` pickles, which can be loaded and applied to new data.

#### 3. Results stored in the database

In the database, `triage` will store results and metadata in several tables. Below is a very brief tour of the most important of these tables.

In the **triage_metadata** schema, you'll find information about your run and the models that were created:
- `triage_metadata.triage_runs`: metadata about every time `triage` is run, identified by a `run_id`
- `triage_metadata.experiments`: configuration information for an experiment, identified by an `experiment_hash`. Note that a config file can be run multiple times, so a specific experiment might be associated with multiple `triage_runs` records. The `experiment_hash` can be linked to the `run_hash` in the `triage_runs` table where `run_type='experiment'`
- `triage_metadata.model_groups`: in `triage` a `model_group` represents a full specification of a model type, set of hyperparameters, set of features, and training set parameters
- `triage_metadata.models`: a `model` represents the application of a `model_group` to a given training set, yielding a set of trained parameters (such as the coefficients of a logistic regression, the splits of a decision tree, etc). The models are identified by both a `model_id` and `model_hash` and can be linked to their `model_group` via the `model_group_id`
- `triage_metadata.experiment_models`: the association between models and experiments (linking an `experiment_hash` to a `model_hash`)

In the **test_results** schema, you'll find information about the validation performance of the models:
- `test_results.evaluations`: performance of each model on the metrics specified in the `scoring` section of your configuration file
- `test_results.predictions`: individual entity-level predicted scores from each model
- `test_results.prediction_metadata`: metadata associated with the predictions
- `test_results.aequitas`: performance of each model on the fairness metrics using the parameters specified in your `bias_audit_config`

In the **train_results** schema, you'll find model performance on the training set, as well as feature importances:
- `train_results.evaluations`: similar to `test_results.evaluations` but for the training set (often may be overfit, but can be useful for debugging)
- `train_results.predictions`: similar to `test_results.predictions` but for the training set
- `train_results.prediction_metadata`: metadata associated with the predictions
- `train_results.feature_importances`: overall feature importances from model training, usining the built-in method for the classifier (if one exists)

Finally, a few intermediate tables can be particularly useful for debugging:
- Tables containing your `cohort` and `label` will be generated in the `public` schema and identified by an associated hash that can be found in your logs.
- The `features` schema contains two types of useful tables: tables containing calculated features for each feature group and "matrix" tables that provide the mapping from each training/validation matrix to `(entity_id, as_of_date)` pairs. Note, however, that these tables may be overwritten if a new run is performed with different feature logic, cohort, or underlying data and should not be assumed to be persistant across runs.

Let's take a quick look at some of these outputs to confirm that our models ran as expected. First, we'll need a database connection...

`triage_metadata.model_groups` should contain four records (for each model type/hyperparameter combination specified in our grid), while `triage_metadata.models` should have twelve (each model group trained on the three validation splits):

In [ ]:
pd.read_sql('SELECT * FROM triage_metadata.model_groups;', db_engine)

In [ ]:
pd.read_sql('SELECT * FROM triage_metadata.models;', db_engine)

You can find the predictions from each model for each project in `test_results.predictions`. Let's take a quick look...

In [ ]:
pd.read_sql('SELECT * FROM test_results.predictions LIMIT 5;', db_engine)

In [ ]:
pd.read_sql("""
  SELECT 
    model_id, 
    COUNT(*) AS num_preds, 
    COUNT(DISTINCT entity_id) AS distinct_entities, 
    AVG(label_value) AS non_funded_rate
  FROM test_results.predictions 
  GROUP BY 1
  ORDER BY 1;
  """, db_engine)

Note that the `label_value` column here is the actual label (e.g., if the project actually failed to be fully funded in four months).

If we want to see how the model performed on your evaluation metrics, you can find these results calculated in `test_results.evaluations`. Let's look at how our models did in term of precision at the top 10% since that reflects our deployment setting:

In [ ]:
pd.read_sql("""
  SELECT *
  FROM test_results.evaluations 
  WHERE metric='precision@' AND parameter='10_pct'
  ORDER BY model_id;
  """, db_engine)

Finally, if you need to work with the training/validation matrices generated by triage or the model objects themselves, you can find them in your project path (here, `triage_output`). Let's take a quick look...

In [ ]:
!ls triage_output/

In [ ]:
!ls -la triage_output/matrices/


In [ ]:
# clean up the database connection
db_engine.dispose()

## Model Selection

`triage` includes a component called `audition` that can help you visualize your model results over time and narrow down your best-performing models. Here we'll provide a quick introduction, but you can find more depth in the [audition tutorial](https://github.com/dssg/triage/blob/master/src/triage/component/audition/Audition_Tutorial.ipynb) as well as the [audition documentation](https://dssg.github.io/triage/audition/audition_intro/) and [model selection concepts overview](https://dssg.github.io/triage/audition/model_selection/). In general, `audition` is best run using a notebook to iteratively explore and narrow down your models.

### Audition Parameters

To run `audition`, you'll need to specify a few parameters:

`metric` and `parameter` together specify the evaluation metric of interest for your project. Note that these need to be calculated as part of the scoring section in your `triage` config and should match the values in the columns of the same name in `test_results.evaluations`

The `run_hash` is an identifier for the run with your complete model grid that you want to evaluate -- the easiest way to find this is from the `triage_metadata.triage_runs` table. This will likely be the `run_hash` associated with the most recent record in that table, but you should be able to figure out which run you want to use from there.

In [ ]:
from triage.component.audition import Auditioner
from triage.component.audition.pre_audition import PreAudition
from triage.component.audition.rules_maker import SimpleRuleMaker, RandomGroupRuleMaker, create_selection_grid

from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

import yaml
from sqlalchemy.engine.url import URL
from triage.util.db import create_engine

import logging
logging.basicConfig(level=logging.WARNING)

import pandas as pd
pd.set_option('display.precision', 4)

In [ ]:
metric = 'precision@'
parameter = '10_pct'
run_hash = '98112c011d842c43e841c415116ef179'

In [ ]:
dbconfig = yaml.safe_load(database_yaml)
db_url = URL(
            'postgres',
            host=dbconfig['host'],
            username=dbconfig['user'],
            database=dbconfig['db'],
            password=dbconfig['pass'],
            port=dbconfig['port'],
        )

conn = create_engine(db_url)

In [ ]:
# table where audition results will be stored
best_dist_table = 'audition_best_dist'

### Pre-Audition: Models and Temporal Splits

Because you may have run several experiments as you iterate, explore, and debug, `audition` needs to know which set of model groups and temporal validation splits to focus on for model selection. While you can specify these directly, `triage` also provides some `pre-audition` tools to help define these.

For example, `get_model_groups_from_experiment()` and `get_train_end_times()` (note that this will return the `train_end_times` associated with the set of model groups returned by one of the `get_model_groups` methods, so those should be run first). Note that the `baseline_model_types` parameter in the constructor is optional and can be used to identify model groups as baselines rather than candidates for model selection.

In [ ]:
pre_aud = PreAudition(
    conn, 
    baseline_model_types=[
        'sklearn.dummy.DummyClassifier',
        'triage.component.catwalk.baselines.rankers.BaselineRankMultiFeature',
        'triage.component.catwalk.baselines.thresholders.SimpleThresholder'
    ]
)

# select model groups by experiment hash id
model_groups = pre_aud.get_model_groups_from_experiment(run_hash)

# Note that this will find train_end_times associated with the model groups defined above
end_times = pre_aud.get_train_end_times(after='1900-01-01')

`get_model_groups_from_experiment()` returns a dictionary with keys `model_groups` and `baseline_model_groups`.

How many of each did we get?

In [ ]:
# Number of non-baseline model groups:
print(len(model_groups['model_groups']))

In [ ]:
# Number of baseline model groups:
print(len(model_groups['baseline_model_groups']))

`get_train_end_times()` returns a list of `train_end_times`:

In [ ]:
end_times

### Setting Up Your Auditioner

`Auditioner` is the main API to do the rules selection and model groups selection. It filters model groups using a two-step process.

- Broad thresholds to filter out truly bad models
- A selection rule grid to find the best model groups over time for each of a variety of methods

Note that model groups that don't have a full set of `train_end_time` splits associated with them will be excluded from the analysis, so **it's important to ensure that all model groups have been completed across all train/test splits**

When we set up our auditioner object, we need to give it a database connection, the model groups to consider (and optionally baseline model groups), train_end_times, and tell it how we're going to filter the models. Note that the `initial_metric_filters` parameter specified below tells `Auditioner` what metric and parameter we'll be using and starts off without any initial filtering constraints (which is what you'll typically want):

In [ ]:
aud = Auditioner(
    db_engine = conn,
    model_group_ids = model_groups['model_groups'],
    train_end_times = end_times,
    initial_metric_filters = [{'metric': metric, 'parameter': parameter, 'max_from_best': 1.0, 'threshold_value': 0.0}],
    distance_table = best_dist_table,
    baseline_model_group_ids = model_groups['baseline_model_groups'] # optional
)

### Using Audition for Model Selection

We can use the `plot_model_groups` method to visualize the performance of our model groups over temporal split (note that the plot may take a few minutes to generate). When this method is called, it applies the metric filters specificied to get rid of really bad model groups with respect the metric of interest. A model group is discarded if:

- It’s never close to the “best” model (based on the `max_from_best` filter) or
- If it’s metric is below a certain number (based on the `threshold_value` filter) at least once

As a starting point, we don't filter out any models, but can iteratively narrow our grid by refining these filters. Let's take a look at our models:

In [ ]:
aud.plot_model_groups()

With our first default setting, we don't filter out models because `max_from_best=1.0` and `threshold_value=0.0` are the loosest criteria.

- The first graph shows us the fraction of models worse than the best model by distance with respect to the metric of interest.
- The second graph shows us the performance of a model group over time. The dashed line is the best case at that time period.

In [ ]:
# model groups that conform to the current filters
aud.thresholded_model_group_ids

We won't delve further into the model selection process here, but see the resources noted above for a deeper look.

## Bias Audit

Finally, we turn to the results of the bias audit using the toolkit [aequitas](http://www.datasciencepublicpolicy.org/our-work/tools-guides/aequitas/), which can be found in the table `test_results.aequitas`:

In [ ]:
pd.read_sql("""SELECT * FROM test_results.aequitas LIMIT 5""", conn)

For each model, the table contains the full set of confusion matrix values for the thresholds and each attribute value specified in your `bias_audit_config`, as well as disparities relative to reference group identified in that configuration file. Note that these disparities are calculated as ratios with the reference group as the denominator.

Audition provides some tools for visualizing these results. If we assume the appropriate fairness metric for this context is true positive rate (TPR) disparity, let's take a look at how model 12 performed at the 10% threshold:

In [ ]:
import triage.component.postmodeling.fairness.aequitas_utils as au
import aequitas.plot as ap

# Load Data
bdf = au.get_aequitas_results(conn, parameter = "0.1_pct", model_id = 12)

# attribute of interest
attribute = 'teacher_prefix'

# fairness metric we care about
metrics = ['tpr']

# tolerance for disparities
disparity_tolerance = 1.30

The `disparity` plot shows disparities for each group on the selected metric relative to the reference group:

In [ ]:
ap.disparity(bdf, metrics, attribute, fairness_threshold = disparity_tolerance)

The `absolute` plot will show the value of the true positive rate itself:

In [ ]:
ap.absolute(bdf, metrics, attribute, fairness_threshold = disparity_tolerance)

We might also want to look at the disparities across models and how they relate to our metric for evaluating model accuracy to consider trade-offs between these goals, which we can do by joining the aequitas table to the evaluations:

In [ ]:
acc_metric = 'precision@'
disp_metric = 'tpr_disparity'
acc_thresh = '10_pct'
disp_thresh = '0.1_pct'
attribute_name = 'teacher_prefix'
attribute_value = 'Mrs.'

df = pd.read_sql(f"""
  WITH last_end_time AS (
    SELECT max(train_end_time) AS train_end_time
    FROM triage_metadata.models
  )
  , last_models AS (
    SELECT model_id
    FROM triage_metadata.models
    JOIN last_end_time USING(train_end_time)
  )
  , eval_metrics AS (
    SELECT e.model_id, e.stochastic_value
    FROM test_results.evaluations e
    JOIN last_models m USING(model_id)
    WHERE e.metric = '{acc_metric}' AND e.parameter = '{acc_thresh}'
  )
  , fair_metrics AS (
    SELECT a.model_id, a.{disp_metric}
    FROM test_results.aequitas a
    JOIN last_models m USING(model_id)
    WHERE a.parameter = '{disp_thresh}'
      AND a.attribute_name = '{attribute_name}'
      AND a.attribute_value = '{attribute_value}'
      AND tie_breaker = 'worst'
  )
  SELECT * 
  FROM eval_metrics e
  JOIN fair_metrics f USING(model_id)
  ORDER BY e.stochastic_value DESC
  """, conn)

df

In [ ]:
df.plot('stochastic_value', disp_metric, kind='scatter')

This final plot helps us visualize the fairness-accuracy trade-offs across the grid of models we ran, with the disparity metric on the y-axis (values closer to 1 don't favor either group) and the accuracy metric on the x-axis (higher values being more accurate).

Although you might choose a model from this set based on these trade-offs, you might also want to consider some of the many fairness-enhancing methods that have been developed for machine learning recently. For an introduction, you may want to explore the materials from our [hands-on fairness tutorial](https://dssg.github.io/fairness_tutorial/).

## Where Do I Go From Here?

You've reached the end of this online `triage` tutorial, but have a few potential options for where to go next:
- Feel free to tinker with the parameters, features, and outputs of the DonorsChoose example here for a working sandbox to start from (keeping in mind the resource constraints of colab, of course)
- For a more comprehensive tutorial that explores `triage` concepts and functionality in much more depth, see our [Dirty Duck Tutorial](https://dssg.github.io/triage/dirtyduck/)
- If you're ready to get started with using `triage` for your own project, check out our [Quickstart Guide](https://dssg.github.io/triage/quickstart/)